In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import nn
from torch import Tensor
from PIL import Image
from torchvision.transforms import Compose, Resize, ToTensor
from einops import rearrange, reduce, repeat
from einops.layers.torch import Rearrange, Reduce
from torchsummary import summary
from torch.utils.data import Dataset,DataLoader
from utils import *
from sklearn import preprocessing as p
import random
import os 
import cv2
from sklearn.preprocessing import normalize,minmax_scale

ATTS = np.load("test2_atts_10.npy",allow_pickle=True)
pred_resp = np.load("test2_pred_resp.npy",allow_pickle=True)
val_img = np.load('../all_sites_data_prepared/pics_data/val_img_m1s1.npy')
val_resp = np.load('../all_sites_data_prepared/New_response_data/valRsp_m1s1.npy')
val_img = np.reshape(val_img,(1000,1,50,50))


In [ ]:
print(len(ATTS))
print(ATTS[0].shape)
print(ATTS[0][0].shape)
print(ATTS[0][0][0].shape)
print(val_resp.shape)
print(pred_resp.shape)


In [ ]:
figure, axis = plt.subplots(5, 5, figsize=(20,20))

for neuron in range(25):
    im = axis[neuron//5,neuron%5].imshow(ATTS[5][neuron].detach().cpu().numpy(), cmap='hot', interpolation='nearest')
    axis[neuron//5,neuron%5].set_title("image number: " + str(neuron))
plt.show()

In [ ]:
figure, axis = plt.subplots(5, 5, figsize=(20,20))

for neuron in range(25):
    im = axis[neuron//5,neuron%5].imshow(ATTS[5][neuron][0].resize(9,9).detach().cpu().numpy(), cmap='hot', interpolation='nearest')
    axis[neuron//5,neuron%5].set_title("image number: " + str(neuron))
plt.show()

In [ ]:
from sklearn.preprocessing import normalize
figure, axis = plt.subplots(5, 5, figsize=(20,20))

for neuron in range(25):
    temp = ATTS[5][neuron][0].reshape(9,9)
    resized = cv2.resize(temp.detach().cpu().numpy(), dsize=(50, 50), interpolation=cv2.INTER_LINEAR) #sum goes from 1 to 30 when u do this
    resized = resized.flatten()
    resized = normalize(resized[:,np.newaxis],axis=0,norm='l1',).ravel()
    resized = resized.reshape(50,50)
    im = axis[neuron//5,neuron%5].imshow(resized, cmap='hot', interpolation='nearest') 
    axis[neuron//5,neuron%5].set_title("image number: " + str(neuron))
plt.show()

In [ ]:
# scaled hm / original img / pp multiplication

neuron_num = 5
image_num = 417

temp = ATTS[neuron_num][image_num][0].resize(9,9).detach().cpu().numpy()
resized = cv2.resize(temp, dsize=(50, 50), interpolation=cv2.INTER_LINEAR) #sum goes from 1 to 30 when u do this
resized = resized.flatten()
resized = normalize(resized[:,np.newaxis],axis=0,norm='l1',).ravel()
resized = resized.reshape(50,50)

figure, axis = plt.subplots(1, 3, figsize=(30,10))

axis[0].imshow(resized,cmap='hot', interpolation='nearest')
axis[1].imshow(val_img[image_num][0],cmap="gray")
temp = val_img[0][0]
temp = np.multiply(temp,resized)
axis[2].imshow(temp,cmap="gray")

plt.show()


In [ ]:
#look at EVERYTHING the neuron pays attention to (binarized)

from sklearn.preprocessing import normalize,minmax_scale

neuron_num = 5
image_num = 417

actual_img = val_img[image_num].reshape(50,50,1)
actual_img = cv2.merge([actual_img,actual_img,actual_img])

temp = ATTS[neuron_num][image_num][0].reshape(9,9).detach().cpu().numpy()
resized = cv2.resize(temp, dsize=(50, 50), interpolation=cv2.INTER_LINEAR) #sum goes from 1 to 30 when u do this
resized = np.power(resized,1.45)
img_bgr = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)
lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
a_component = lab[:,:,0] #brightest part maybe interesting?
blur = cv2.GaussianBlur(a_component,(9,9), 11)
blur = minmax_scale(blur.ravel(),feature_range=(0,255)).reshape(50,50)
blur = blur.astype(np.uint8)
heatmap_img = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
for i in range(50):
    for j in range(50):
        if np.array_equiv(heatmap_img[i][j],np.array([128,0,0])):
            heatmap_img[i][j] = np.array([0,0,0])

spi_img = heatmap_img * (0.001) + actual_img * (1)
print(heatmap_img[0].shape)

plt.imshow(spi_img)

In [ ]:
# #look at EVERYTHING the neuron pays attention to (binarized)

# from sklearn.preprocessing import normalize,minmax_scale

# neuron_num = 5
# image_num = 996

# actual_img = val_img[image_num].reshape(50,50,1)
# # actual_img = cv2.cvtColor(actual_img,cv2.COLOR_GRAY2RGB)
# actual_img = cv2.merge([actual_img,actual_img,actual_img])

# temp = ATTS[neuron_num][image_num][0].reshape(9,9).detach().cpu().numpy()
# resized = cv2.resize(temp, dsize=(50, 50), interpolation=cv2.INTER_LINEAR) #sum goes from 1 to 30 when u do this
# # resized = minmax_scale(resized.ravel(), feature_range=(0,255)).reshape(50,50) #needed?
# resized = np.power(resized,1.45)
# # plt.show()
# img_bgr = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)
# lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
# a_component = lab[:,:,0] #brightest part maybe interesting?
# # th = cv2.threshold(lab,1,255,cv2.THRESH_BINARY)[1] #1 is always ok?
# blur = cv2.GaussianBlur(a_component,(9,9), 11)
# # print(blur[20])
# blur = minmax_scale(blur.ravel(),feature_range=(0,255)).reshape(50,50)
# blur = blur.astype(np.uint8)
# # print(blur[0])
# heatmap_img = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
# for i in range(50):
#     for j in range(50):
#         if np.array_equiv(heatmap_img[i][j],np.array([128,0,0])):
#             heatmap_img[i][j] = np.array([0,0,0])

# spi_img = heatmap_img * (0.001) + actual_img * (1)
# print(heatmap_img[0].shape)

# plt.imshow(spi_img)
# #https://stackoverflow.com/questions/46020894/superimpose-heatmap-on-a-base-image-opencv-python

In [ ]:
# generate image as : original / scaled heatmap / opencv conversion heatmap / weighted sum output

neuron_num = 5
image_num = 10

actual_img = val_img[image_num].reshape(50,50,1)
actual_img = cv2.merge([actual_img,actual_img,actual_img])

temp = ATTS[neuron_num][image_num][0].reshape(9,9).detach().cpu().numpy()
resized = cv2.resize(temp, dsize=(50, 50), interpolation=cv2.INTER_LINEAR)
resized = np.power(resized,1.45)
img_bgr = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)
lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
a_component = lab[:,:,0]
blur = a_component
blur = minmax_scale(blur.ravel(),feature_range=(0,255)).reshape(50,50)
blur = blur.astype(np.uint8)
heatmap_img = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
for i in range(50):
    for j in range(50):
        if np.array_equiv(heatmap_img[i][j],np.array([128,0,0])):
            heatmap_img[i][j] = np.array([0,0,0])

spi_img = heatmap_img * (0.001) + actual_img * (1)

figure, axis = plt.subplots(1, 4, figsize=(40,10))
figure.suptitle("Real Neuron Number: " + str(neuron_num) + ", Image Number: " + str(image_num)+ "\nActual Neural Response: " + str(val_resp[image_num][neuron_num]) + ", Predicted Neural Response: " + str(pred_resp[image_num][neuron_num]),fontsize=36)
axis[0].title.set_text('Original Image')
axis[1].title.set_text('Interpolated Heatmap (Greyscale - Displayed with Color)')
axis[2].title.set_text('Colored Heatmap (RGB)')
axis[3].title.set_text('Superimposed Image (Weighted Sum)')
axis[0].imshow(actual_img)
axis[1].imshow(resized,cmap='hot', interpolation='nearest')
axis[2].imshow(heatmap_img)
axis[3].imshow(spi_img)


In [ ]:
#OK PAST THIS WILL BE FOR SAVING GENERATED IMAGES

In [ ]:
def save_image(neuron_num,image_num,save_path):
    actual_img = val_img[image_num].reshape(50,50,1)
    actual_img = cv2.merge([actual_img,actual_img,actual_img])

    temp = ATTS[neuron_num][image_num][0].reshape(9,9).detach().cpu().numpy()
    resized = cv2.resize(temp, dsize=(50, 50), interpolation=cv2.INTER_LINEAR)
    resized = np.power(resized,1.45)
    img_bgr = cv2.cvtColor(resized, cv2.COLOR_GRAY2BGR)
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    a_component = lab[:,:,0]
    blur = a_component
    blur = minmax_scale(blur.ravel(),feature_range=(0,255)).reshape(50,50)
    blur = blur.astype(np.uint8)
    heatmap_img = cv2.applyColorMap(blur, cv2.COLORMAP_JET)
    for i in range(50):
        for j in range(50):
            if np.array_equiv(heatmap_img[i][j],np.array([128,0,0])):
                heatmap_img[i][j] = np.array([0,0,0])

    spi_img = heatmap_img * (0.001) + actual_img * (1)

    figure, axis = plt.subplots(1, 4, figsize=(40,10))
    figure.suptitle("Real Neuron Number: " + str(neuron_num) + ", Image Number: " + str(image_num)+ "\nActual Neural Response: " + str(val_resp[image_num][neuron_num]) + ", Predicted Neural Response: " + str(pred_resp[image_num][neuron_num]),fontsize=36)
    axis[0].title.set_text('Original Image')
    axis[1].title.set_text('Interpolated Heatmap (Greyscale - Displayed with Color)')
    axis[2].title.set_text('Colored Heatmap (RGB)')
    axis[3].title.set_text('Superimposed Image (Weighted Sum)')
    axis[0].imshow(actual_img)
    axis[1].imshow(resized,cmap='hot', interpolation='nearest')
    axis[2].imshow(heatmap_img)
    axis[3].imshow(spi_img)
    plt.savefig(save_path)

In [ ]:
test_number = 8
ATTS = np.load("test"+str(test_number)+"_atts_10.npy",allow_pickle=True)
pred_resp = np.load("test"+str(test_number)+ "_pred_resp.npy",allow_pickle=True)
val_img = np.load('../all_sites_data_prepared/pics_data/val_img_m1s1.npy')
val_resp = np.load('../all_sites_data_prepared/New_response_data/valRsp_m1s1.npy')
val_img = np.reshape(val_img,(1000,1,50,50))
curr_dir = os.path.dirname("new_att_vis.ipynb")

for neuron_num in range(5,10):
    save_dir = os.path.join(curr_dir, f'./simp_img/test{test_number}_neuron_{neuron_num}/')
    if not os.path.isdir(save_dir):
            os.makedirs(save_dir)
    for image_num in range(30,1000):
        file_name = f'neuron_{neuron_num}_image_{image_num}'
        save_path = save_dir+file_name
        save_image(neuron_num,image_num,save_path)